### 기본 세팅

In [1]:
import time
from datetime import date, timedelta
import urllib
import pandas as pd

from selenium import webdriver

In [2]:
# 어제 날짜 저장
def get_yesterday():
    today      = date.today()             # 오늘 날짜
    yesterday1 = today - timedelta(1)     # 어제 날짜
    
    year0 = str(today.year)
    mon0  = str(today.month)
    if int(mon0) < 10:
        mon0 = '0'+str(mon0)
    day0  = str(today.day)
    if int(day0) < 10:
        day0 = '0'+str(day0)

    year1 = str(yesterday1.year)
    mon1  = str(yesterday1.month)
    if int(mon1) < 10:
        mon1 = '0'+str(mon1)
    day1  = str(yesterday1.day)
    if int(day1) < 10:
        day1 = '0'+str(day1)

    return year0+'.'+mon0+'.'+day0, year1+'.'+mon1+'.'+day1

### 네이버 종토방 크롤링

In [3]:
def get_jongto_info(keyword):
    
    # 크롤링 정보(크롤링 날짜, 종목코드)
    today, yesterday1 = get_yesterday()
    stockcode = pd.read_html('http://kind.krx.co.kr/corpgeneral/corpList.do?method=download', header=0)[0]
    stockcode = stockcode[['회사명', '종목코드']]
    stockcode = stockcode.rename(columns={'회사명': 'company', '종목코드': 'code'})
    stockcode.code = stockcode.code.map('{:06d}'.format)
    
    # 검색종목-검색어: 삼성전자-삼성전자, 삼성전자-삼전, 셀트리온-셀트
    code     = stockcode.loc[stockcode.company==keyword[0]]['code'].values.tolist()[0]     # 검색종목
    encoding = urllib.parse.quote(keyword[1], encoding='euc-kr')                           # 검색어
    
    
    
    # 드라이버 실행
    driver = webdriver.Chrome('chromedriver.exe')
    driver.implicitly_wait(3)
    sleeptime = 0.5     # 프로세스 일시정지 시간
    
    
    
    page    = 0
    page_y1 = 0        # 맨위 게시날짜가 어제인 페이지
    page_y2 = 0        # 맨위 게시날짜가 그저께인 페이지
    endpage = 0        # 더이상 보지않을 페이지
    
    # 페이지 특정해 크롤링
    while page < 50:
        try:
            # 페이지 이동
            page += 1
            driver.get(f'https://finance.naver.com/item/board.nhn?code={code}&st=title&sw={encoding}&page={page}')
            time.sleep(sleeptime)        

            
            
            # 게시날짜 크롤링
            current = driver.find_elements_by_css_selector('#content > div > table > tbody > tr > td > span')
            top_date = current[0].text
            
            # 맨위 게시날짜가 오늘, 어제가 아닌 경우 마지막 페이지로 지정
            if today not in top_date and yesterday1 not in top_date:
                endpage = page
                break
        except:
            pass

    
    
    list_date        = []     # 게시날짜 리스트
    list_address     = []     # 게시글주소 리스트
    list_address_yes = []     # 어제 게시글주소 리스트

    # 게시날짜가 어제인 경우만 골라서 크롤링
    for page in range(1, endpage, 1):
        try:
            # 페이지 이동
            driver.get(f'https://finance.naver.com/item/board.nhn?code={code}&st=title&sw={encoding}&page={page}')
            time.sleep(sleeptime)

            
            
            # 게시날짜 크롤링
            current = driver.find_elements_by_css_selector('#content > div > table > tbody > tr > td > span')
            cnt = 0
            for date in current:
                if cnt%2 == 1:                                                             # 예외처리(조회수 수집안함)
                    cnt += 1
                    continue
                list_date.append(date.text)
                cnt += 1
                                                   
                
                
            # 게시글주소 크롤링
            current = driver.find_elements_by_css_selector('#content > div > table > tbody > tr > td > a')
            for address in current:
                cur_address = address.get_attribute('href')
                if ('javascript' not in cur_address) and ('mode' not in cur_address):     # 예외처리(게시글 이외의 주소)
                    list_address.append(address.get_attribute('href')) 
        except:
            pass

    # 어제 뉴스주소만 저장
    for i in range(len(list_date)):
        if yesterday1 in list_date[i]:
            list_address_yes.append(list_address[i])
    
    
    
    list_headline    = []     # 제목 리스트
    list_content     = []     # 본문 리스트
    list_comment     = []     # 댓글 리스트
    list_comment_num = []     # 댓글개수 리스트

    # 어제 게시글 주소를 돌며 크롤링
    for url in list_address_yes:
        try:
            # 페이지 이동
            driver.get(url)
            time.sleep(sleeptime)

            
            
            # 제목 크롤링
            current = driver.find_elements_by_css_selector('#content > div > table > tbody > tr > th > strong')
            for headline in current:
                if len(headline.text) != 1 and not headline.text.isdigit():               # 예외처리
                    list_headline.append(headline.text)

                    
                    
            # 본문 크롤링
            current = driver.find_elements_by_css_selector('#body')     
            for content in current:
                list_content.append(content.text)

                
                
            # 댓글 크롤링
            current = driver.find_elements_by_css_selector('#cbox_module_wai_u_cbox_content_wrap_tabpanel > ul > li > div > div > div > span')
            cur_comment = []
            cnt = 0
            for comment in current:
                if cnt%5 == 2:                                                             # 예외처리(댓글 내용만 가져옴)
                    cur_comment.append(comment.text)     
                cnt += 1
            list_comment.append(cur_comment)
        except:
            pass
    driver.close()
        
    # 댓글개수 저장
    for i in range(len(list_comment)):
        list_comment_num.append(len(list_comment[i]))
          
    
    
    comment_headline = []     # 댓글모을 제목 리스트
    comment_comment  = []     # 제목별 댓글 리스트

    # 제목별 댓글 모음
    for i in range(len(list_comment_num)):
        cur_headline = list_headline[i]

        # 댓글 없는 경우 해당 안됨
        if int(list_comment_num[i]) == 0:
            continue

        for j in range(len(list_comment[i])):
            cur_comment = list_comment[i][j]
            comment_headline.append(cur_headline)
            comment_comment.append(cur_comment)
        
        
        
    jongto_info = pd.DataFrame({'제목': list_headline, '주소': list_address_yes, '본문': list_content})
    jongto_comment = pd.DataFrame({'제목': comment_headline, '댓글': comment_comment})        
    return jongto_info, jongto_comment

### 크롤링 예시

In [4]:
jongto_대한항공_info, jongto_대한항공_comment = get_jongto_info(['대한항공', '대한항공'])

In [5]:
jongto_대한항공_info

,제목,주소,본문
0,대한항공…,https://finance.naver.com/item/board_read.nhn?...,25000원까지 열려있습니다. 조심하세요\n\n여러분들의 행운을 빕니다.-네이키드콜-
1,대한항공…,https://finance.naver.com/item/board_read.nhn?...,아시아나 급등으로 목표가 수정합니다.\n\n25000원까지 열려있습니다. 조심하세요...
2,대한항공 40000원까지 열려있습니다,https://finance.naver.com/item/board_read.nhn?...,너무싸\n활짝 열려 있습니다\n-네이키드골골-
3,대한항공 존나 각설이네,https://finance.naver.com/item/board_read.nhn?...,죽지도않고 3만 또왔내
4,대한항공…,https://finance.naver.com/item/board_read.nhn?...,19000원까지 열려있습니다. 조심하세요\n\n여러분들의 행운을 빕니다.-네이키드콜-
5,대한항공…,https://finance.naver.com/item/board_read.nhn?...,- 컵라묜 이라도 먹고 하시게요 -
6,대한항공…,https://finance.naver.com/item/board_read.nhn?...,19000원까지 열려있습니다. 조심하세요\n\n여러분들의 행운을 빕니다.-네이키드콜-


In [6]:
jongto_대한항공_comment

,제목,댓글
0,대한항공…,30퍼나 상향해줘서 고맙다 ㅋㅋㅋㅋ
1,대한항공…,19000충 에서 25000충으로 변신
2,대한항공…,클린봇이 부적절한 표현을 감지한 댓글입니다.
3,대한항공…,
4,대한항공…,클린봇이 부적절한 표현을 감지한 댓글입니다.
5,대한항공…,
6,대한항공…,왔냐?밥이나먹자
7,대한항공…,국민항공
8,대한항공…,19000충 출첵
9,대한항공…,zzzzzzz귀여워
